In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from keras.utils import np_utils
from tensorflow.keras.models import Model, Sequential, load_model
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Reshape, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.resnet50 import ResNet50
from sklearn.metrics import confusion_matrix, classification_report
import itertools
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
from keras.applications.vgg16 import VGG16

In [ ]:
from keras.applications.vgg19 import VGG19

In [ ]:
from tensorflow.keras.utils import to_categorical

In [ ]:
# Constants
img_rows = 224
img_cols = 224
input_shape = (img_rows,img_cols,3)
epochs = 10
batch_size = 64


In [ ]:
# Get ResNet-50 Model
def getResNet50Model(lastFourTrainable=False):
    resnet_model = ResNet50(weights='imagenet', input_shape=input_shape, include_top=True)
    # Make all layers non-trainable
    for layer in resnet_model.layers[:]:
        layer.trainable = False
    # Add fully connected layer which have 1024 neuron to ResNet-50 model
    output = resnet_model.get_layer('avg_pool').output
    output = Flatten(name='new_flatten')(output)
    output = Dense(units=1024, activation='relu', name='new_fc')(output)
    predictions = Dense(units=50, activation='softmax')(output)
    resnet_model = Model(resnet_model.input, predictions)
    # Make last 4 layers trainable if lastFourTrainable == True
    if lastFourTrainable == True:
        resnet_model.get_layer('conv5_block3_2_bn').trainable = True
        resnet_model.get_layer('conv5_block3_3_conv').trainable = True
        resnet_model.get_layer('conv5_block3_3_bn').trainable = True
        resnet_model.get_layer('new_fc').trainable = True
    # Compile ResNet-50 model
    resnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    resnet_model.summary()
    return resnet_model

In [ ]:
traindf = pd.read_csv("../input/landmark-recognition-2021/train.csv")
traindf.head()

In [ ]:
traindf.shape

In [ ]:
landmark_unique = traindf['landmark_id'].unique()
len(landmark_unique)

In [ ]:
landmark_unique[0:50]

In [ ]:
image_ids = []
labels = []
temp_labels = []
i=0
for id_ in landmark_unique[0:50]:
    for iid in traindf['id'][traindf['landmark_id'] == id_]:
        image_ids.append(iid)
        labels.append(id_)
        temp_labels.append(i)
    i = i+1
len(image_ids)

In [ ]:
mainpath = '../input/landmark-recognition-2021/train'
image_path = []
images_pixels = []

for i in range(0,len(image_ids)):
    first_dir = os.path.join(mainpath,image_ids[i][0])
    second_dir = os.path.join(first_dir,image_ids[i][1])
    third_dir = os.path.join(second_dir,image_ids[i][2])
    finalpath = os.path.join(third_dir,image_ids[i]+'.jpg')
    
    img_pix = cv2.imread(finalpath,1)
    images_pixels.append(cv2.resize(img_pix, (224,224)))
    
    image_path.append(finalpath)

In [ ]:
print('Images: ', len(image_path))
print('Image labels: ', len(labels))

In [ ]:
print(temp_labels)

In [ ]:
import random

In [ ]:
fig = plt.gcf()
fig.set_size_inches(16, 16)

next_pix_ = image_path

for i, img_path in enumerate(next_pix_[0:16]):
    
    sp = plt.subplot(5, 4, i + 1)
    sp.axis('Off')

    img = cv2.imread(img_path)
    plt.imshow(img)

plt.show()

In [ ]:
fig = plt.gcf()
fig.set_size_inches(16, 16)

next_pix = image_path
random.shuffle(next_pix)

for i, img_path in enumerate(next_pix[0:12]):
    
    sp = plt.subplot(4, 4, i + 1)
    sp.axis('Off')

    img = cv2.imread(img_path)
    plt.imshow(img)

plt.show()

In [ ]:
shuf = list(zip(images_pixels,temp_labels))
random.shuffle(shuf)

train_data, labels_data = zip(*shuf)
print('Images: ', len(train_data))
print('Image labels: ', len(labels_data))

In [ ]:
print(labels_data)

In [ ]:
'''# Function for plotting the Confusion Matrix
def plotConfusionMatrix(cm, class_names, normalize=True, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.figure(figsize=(10,10))
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=45)
    plt.yticks(tick_marks, class_names)
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        cm = np.around(cm, decimals=2)
        cm[np.isnan(cm)] = 0.0
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                horizontalalignment="center",
                color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()
    '''

In [ ]:
"""
# Function for train the given model and get Confusion Matrix
def trainModelAndGetConfusionMatrix(model,train_generator,valid_generator,test_generator,epochs,batch_size):
    # Fit the model
    model.fit_generator(train_generator,
                      epochs=epochs,
                      steps_per_epoch=len(train_generator) // batch_size,
                      validation_data=valid_generator,
                      validation_steps=len(valid_generator) // batch_size)
    # Evaluate the model
    loss_and_metrics = model.evaluate_generator(test_generator, steps=len(test_generator) // batch_size+1)
    print("Test Loss: ", loss_and_metrics[0])
    print("Test Accuracy: ", loss_and_metrics[1])
    test_generator.reset()
    # Get Confusion Matrix and plot it
    Y_pred = model.predict_generator(test_generator, steps=len(test_generator) // batch_size+1)
    y_pred = np.argmax(Y_pred, axis=1)
    class_names = []
    for key in train_generator.class_indices:
        class_names.append(key)
    cr = classification_report(test_generator.classes, y_pred, target_names=class_names)
    print("Classification Report\n" + cr) # Print classification report
    cm = confusion_matrix(test_generator.classes, y_pred)
    plotConfusionMatrix(cm, class_names, normalize=False, title='Confusion Matrix')
    plotConfusionMatrix(cm, class_names, normalize=True, title='Confusion Matrix')
    return model
"""

In [ ]:
from sklearn.preprocessing import LabelBinarizer

In [ ]:
train_data = np.array(train_data) #/ 255


In [ ]:
a = np.array(labels_data)
a_pd = pd.get_dummies(a).astype('float32').values 

In [ ]:
print(labels_data[0:6])

In [ ]:
print(a_pd)

In [ ]:
labels_data=a_pd

In [ ]:
print(labels_data[0:5])

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(train_data, labels_data, test_size = 0.3, random_state=101)

print("X train data : ", len(X_train))
print("X label data : ", len(X_test))
print("Y test data : ", len(Y_train))
print("Y label data : ", len(Y_test))

In [ ]:
print(Y_train[0:5])

In [ ]:
num_sample=X_train.shape[0]
#20% of the enteries has to be in validation set
validation_freq=int(num_sample*0.2)
#Generating random sample of indices equal to validation_freq
validationlist = random.sample(range(0, num_sample), validation_freq)
traininglist=list(set(range(0,num_sample))-set(validationlist))
print("No interesection between validationlist and traininglist:",set(traininglist).intersection(validationlist))

In [ ]:
train_x_final=[]
train_y_final=[]
for val in traininglist:
    train_x_final.append(X_train[val])
    train_y_final.append(Y_train[val])
train_x_final=np.array(train_x_final)
train_y_final=np.array(train_y_final)
print("Training data shape",train_x_final.shape)
print("Training label shape",train_y_final.shape)

In [ ]:
validation_x_final=[]
validation_y_final=[]
for val in validationlist:
    validation_x_final.append(X_train[val])
    validation_y_final.append(Y_train[val])
validation_x_final=np.array(validation_x_final)
validation_y_final=np.array(validation_y_final)
print("Validation data shape",validation_x_final.shape)
print("Validation label shape",validation_y_final.shape)

In [ ]:
validation_y_final[0]

In [ ]:
train_y_final[1]

In [ ]:
# Get VGG-16 Model
def getVGG16Model(lastFourTrainable=False):
    vgg_model = VGG16(weights='imagenet', input_shape=input_shape, include_top=True)
    # Make all layers untrainable
    for layer in vgg_model.layers[:]:
        layer.trainable = False
    # Add fully connected layer which have 1024 neuron to VGG-16 model
    output = vgg_model.get_layer('fc2').output
    output = Flatten(name='new_flatten')(output)
    output = Dense(units=1024, activation='relu', name='new_fc')(output)
    output = Dense(units=50, activation='softmax')(output)
    vgg_model = Model(vgg_model.input, output)
    # Make last 4 layers trainable if lastFourTrainable == True
    if lastFourTrainable == True:
        vgg_model.get_layer('block5_conv3').trainable = True
        vgg_model.get_layer('fc1').trainable = True
        vgg_model.get_layer('fc2').trainable = True
        vgg_model.get_layer('new_fc').trainable = True
    # Compile VGG-16 model
    vgg_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    vgg_model.summary()

    return vgg_model

In [ ]:
# Get VGG-19 Model
def getVGG19Model(lastFourTrainable=False):
    vgg_model_19 = VGG19(weights='imagenet', input_shape=input_shape, include_top=True)
    # Make all layers untrainable
    for layer in vgg_model_19.layers[:]:
        layer.trainable = False
    # Add fully connected layer which have 1024 neuron to VGG-16 model
    output = vgg_model_19.get_layer('fc2').output
    output = Flatten(name='new_flatten')(output)
    output = Dense(units=1024, activation='relu', name='new_fc')(output)
    output = Dense(units=50, activation='softmax')(output)
    vgg_model_19 = Model(vgg_model_19.input, output)
    # Make last 4 layers trainable if lastFourTrainable == True
    if lastFourTrainable == True:
        vgg_model_19.get_layer('block5_conv3').trainable = True
        vgg_model_19.get_layer('fc1').trainable = True
        vgg_model_19.get_layer('fc2').trainable = True
        vgg_model_19.get_layer('new_fc').trainable = True
    # Compile VGG-16 model
    vgg_model_19.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    vgg_model_19.summary()

    return vgg_model_19

In [ ]:
vgg_model19_a = getVGG19Model(lastFourTrainable=False)

In [ ]:
vgg_model19_b = getVGG19Model(lastFourTrainable=True)

In [ ]:
# Get ResNet-50 Model with lastFourTrainable=False
resnet_model_a = getResNet50Model(lastFourTrainable=False)

In [ ]:
from tensorflow.keras.optimizers import Adam

In [ ]:
print(validation_x_final.shape)
print(validation_y_final.shape)

In [ ]:
import tensorflow

In [ ]:
train_x_final=tensorflow.keras.applications.resnet.preprocess_input(train_x_final)
validation_x_final=tensorflow.keras.applications.resnet.preprocess_input(validation_x_final)

In [ ]:
# Train ResNet-50 Model 
#resnet_model_a.compile(optimizer=Adam(learning_rate=0.001),loss='categorical_crossentropy',metrics=['accuracy'])
history = resnet_model_a.fit(train_x_final, train_y_final, epochs=10,validation_data=(validation_x_final, validation_y_final))
#resnet_model_a = trainModelAndGetConfusionMatrix(resnet_model_a,train_x_final,validation_x_final,X_test,10,64)

In [ ]:
vgga=getVGG16Model(lastFourTrainable=False)


In [ ]:
history = vgga.fit(train_x_final, train_y_final,validation_data=(validation_x_final,validation_y_final), epochs=10)


In [ ]:
history_vgg19_a = vgg_model19_a.fit(train_x_final, train_y_final,validation_data=(validation_x_final,validation_y_final), epochs=10)

In [ ]:
history_vgg19_b = vgg_model19_b.fit(train_x_final, train_y_final,validation_data=(validation_x_final,validation_y_final), epochs=20)

In [ ]:
vggb=getVGG16Model(lastFourTrainable=True)

In [ ]:
history = vggb.fit(train_x_final, train_y_final,validation_data=(validation_x_final,validation_y_final), epochs=10)


In [ ]:
resnet_model_b = getResNet50Model(lastFourTrainable=True)

In [ ]:
# Train ResNet-50 Model 
#resnet_model_a.compile(optimizer=Adam(learning_rate=0.001),loss='categorical_crossentropy',metrics=['accuracy'])
history = resnet_model_b.fit(train_x_final, train_y_final, epochs=10,validation_data=(validation_x_final, validation_y_final))
#resnet_model_a = trainModelAndGetConfusionMatrix(resnet_model_a,train_x_final,validation_x_final,X_test,10,64)

DenseNet

In [ ]:
from keras.applications.densenet import DenseNet121

In [ ]:
# Get DenseNet-121 Model
def getDenseNet121Model(lastFourTrainable=False):
    densenet_model = DenseNet121(weights='imagenet', input_shape=input_shape, include_top=True)
    # Make all layers non-trainable
    for layer in densenet_model.layers[:]:
        layer.trainable = False
    # Add fully connected layer which have 1024 neuron to ResNet-50 model
    output = densenet_model.get_layer('avg_pool').output
    output = Flatten(name='new_flatten')(output)
    output = Dense(units=1024, activation='relu', name='new_fc')(output)
    predictions = Dense(units=50, activation='softmax')(output)
    densenet_model = Model(densenet_model.input, predictions)
    # Make last 4 layers trainable if lastFourTrainable == True
    if lastFourTrainable == True:
        densenet_model.get_layer('conv5_block3_2_bn').trainable = True
        densenet_model.get_layer('conv5_block3_3_conv').trainable = True
        densenet_model.get_layer('conv5_block3_3_bn').trainable = True
        densenet_model.get_layer('new_fc').trainable = True
    # Compile ResNet-50 model
    densenet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    densenet_model.summary()
    return densenet_model

In [ ]:
# Get DenseNet-121 Model with lastFourTrainable=False
densenet_model_a = getDenseNet121Model(lastFourTrainable=False)

In [ ]:
# Train ResNet-50 Model 
#resnet_model_a.compile(optimizer=Adam(learning_rate=0.001),loss='categorical_crossentropy',metrics=['accuracy'])
history_densenet_model_a = densenet_model_a.fit(train_x_final, train_y_final, epochs=10,validation_data=(validation_x_final, validation_y_final))
#resnet_model_a = trainModelAndGetConfusionMatrix(resnet_model_a,train_x_final,validation_x_final,X_test,10,64)

In [ ]:
# plot the training results
import matplotlib.pyplot as plt

acc = history_densenet_model_a.history['accuracy']
val_acc = history_densenet_model_a.history['val_accuracy']
loss = history_densenet_model_a.history['loss']
val_loss = history_densenet_model_a.history['val_loss']

epochs = range(1, len(acc)+1)

plt.plot(epochs, acc, '#21466C', label='Training acc')
plt.plot(epochs, val_acc, '#ff0051', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, '#21466C', label='Training loss')
plt.plot(epochs, val_loss, '#ff0051', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

## KNN- Image Retrieval

In [ ]:
from sklearn.neighbors import KNeighborsClassifier as kNN